In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

In [5]:
from keras.layers import LSTM
from keras.utils import np_utils

Load Data
===

In [6]:
data = pd.read_csv('/home/atharva/Downloads/PROJECT/StdData.csv',parse_dates=['DATE'])
data = data.ix[:,'DATE':]

Training and Testing Data
===

In [7]:

from sklearn.model_selection import StratifiedShuffleSplit

In [8]:
split = StratifiedShuffleSplit(n_splits = 3 , test_size = 0.3, random_state = 0 )
for train_index,test_index in split.split(data,data['Cat']):
    train = data.loc[train_index]
    test = data.loc[test_index]
train.index = range(len(train))
test.index = range(len(test))

In [9]:
train.columns

Index(['DATE', 'Month', '..MAX', '..MIN', 'AW', 'SLP', 'MSP', 'DPT', 'RH',
       'VP', 'Cat', '..R.F'],
      dtype='object')

In [26]:
from sklearn import preprocessing

In [28]:
std_scale = preprocessing.StandardScaler().fit(train.ix[:,'..MAX':'VP'])
train[['..MAX','..MIN','AW','SLP','MSP','DPT','RH','VP']]= std_scale.transform(train.ix[:,'..MAX':'VP'])

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
np.random.seed(7)

In [12]:
model = Sequential()
model.add(Dense(32,input_shape=(20,)))
model.add(LSTM(3,init="uniform",inner_init="glorot_uniform",activation="sigmoid"))
model.add(Dense(35,1,activation="linear"))
model.compile(loss='mean_absolute_error', optimizer='rmsprop')


/home/atharva/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(3, kernel_initializer="uniform", recurrent_initializer="glorot_uniform", activation="sigmoid")`
  app.launch_new_instance()


ValueError: Input 0 is incompatible with layer lstm_3: expected ndim=3, found ndim=2

In [15]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	
# reshape X to be [samples, time steps, features]
np.reshape(std, (len(dataX), 1, 1))


array([[[ 0]],

       [[ 1]],

       [[ 2]],

       [[ 3]],

       [[ 4]],

       [[ 5]],

       [[ 6]],

       [[ 7]],

       [[ 8]],

       [[ 9]],

       [[10]],

       [[11]],

       [[12]],

       [[13]],

       [[14]],

       [[15]],

       [[16]],

       [[17]],

       [[18]],

       [[19]],

       [[20]],

       [[21]],

       [[22]],

       [[23]],

       [[24]]])

In [17]:
data.columns

Index(['DATE', 'Month', '..MAX', '..MIN', 'AW', 'SLP', 'MSP', 'DPT', 'RH',
       'VP', 'Cat', '..R.F'],
      dtype='object')

In [19]:
data2  = data.ix[:,'..MAX':'VP']

In [21]:
data2 = pd.concat([data2,pd.get_dummies(data['Month'])],axis =1)

In [24]:
data2.shape[1]

20

In [26]:
 np.reshape(np.array(data2),(len(np.array(data2)),2,data2.shape[1]))

ValueError: total size of new array must be unchanged

In [40]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset.ix[i:(i+look_back), '..MAX':'VP']
		dataX.append(a)
		dataY.append(dataset.ix[i + look_back, '..R.F'])
	return dataX, dataY


In [41]:
data.columns

Index(['DATE', 'Month', '..MAX', '..MIN', 'AW', 'SLP', 'MSP', 'DPT', 'RH',
       'VP', 'Cat', '..R.F'],
      dtype='object')

In [42]:
dataXX, dataYY = create_dataset(data,2)

[      ..MAX     ..MIN        AW       SLP       MSP       DPT        RH  \
 0 -1.123253 -1.516076 -0.182702  0.040799  0.512949 -1.723437 -0.963904   
 1 -1.499083 -1.815405  0.178337  0.243996  0.682997 -1.760522 -0.870464   
 2 -1.273585 -2.014957 -0.363222  0.255282  0.699187 -1.360007 -0.963904   
 
          VP  
 0 -1.959742  
 1 -2.148232  
 2 -2.308798  ,
       ..MAX     ..MIN        AW       SLP       MSP       DPT        RH  \
 1 -1.499083 -1.815405  0.178337  0.243996  0.682997 -1.760522 -0.870464   
 2 -1.273585 -2.014957 -0.363222  0.255282  0.699187 -1.360007 -0.963904   
 3 -1.273585 -0.997239 -0.543742  0.255282  0.602025 -1.182000 -0.777024   
 
          VP  
 1 -2.148232  
 2 -2.308798  
 3 -2.273893  ,
       ..MAX     ..MIN        AW       SLP       MSP       DPT        RH  \
 2 -1.273585 -2.014957 -0.363222  0.255282  0.699187 -1.360007 -0.963904   
 3 -1.273585 -0.997239 -0.543742  0.255282  0.602025 -1.182000 -0.777024   
 4 -1.073143 -0.478402 -0.724262  0.37